In [ ]:
# Setup environment
using Pkg
Pkg.activate("..")

ENV["RAD_DIR"] = abspath("..", "socrates")

using Revise
include("../src/AGNI.jl")

using Printf
using Plots

In [ ]:
# Common parameters
ROOT_DIR = abspath("..")
OUT_DIR_BASE = abspath("..", "out", "test_deep_heating")
spectral_file = "res/spectral_files/Dayspring/48/Dayspring.sf"
star_file = "res/stellar_spectra/sun.txt"

# Planet parameters (hot Jupiter-like)
instellation = 5000.0      # W/m² (moderate stellar flux)
tmp_surf = 1500.0          # K
gravity = 10.0             # m/s²
radius = 7.0e7             # m (roughly Jupiter radius)
p_surf = 100.0             # bar
p_top = 1e-5               # bar
nlev = 100                 # number of levels

# Deep heating parameters
deep_P_dep = 1.0e6         # 10 bar deposition pressure
deep_sigma_P = 1.0         # Width in log-pressure
deep_efficiency = 0.02     # 2% of stellar flux

# Expected additional flux
F_deep_expected = deep_efficiency * instellation
@printf("Expected deep heating flux: %.2f W/m²\n", F_deep_expected)

In [ ]:
# Function to create and run an atmosphere model
function run_model(out_dir::String, deep_active::Bool)
    # Clean output directory
    rm(out_dir, force=true, recursive=true)
    mkpath(out_dir)

    # Create atmosphere
    atmos = AGNI.atmosphere.Atmos_t()

    # Composition: H2-dominated with some H2O
    mf_dict = Dict("H2" => 0.85, "He" => 0.14, "H2O" => 0.01)

    # Setup atmosphere
    AGNI.atmosphere.setup!(atmos, ROOT_DIR, out_dir,
                           spectral_file,
                           instellation, 1.0, 0.0, 48.19,  # instellation, s0_fact, albedo_b, zenith
                           tmp_surf, gravity, radius,
                           nlev, p_surf, p_top,
                           mf_dict, "";  # mf_dict, mf_path
                           flag_rayleigh=false,
                           flag_gcontinuum=true,
                           real_gas=false,
                           thermo_functions=true)

    # Allocate
    AGNI.atmosphere.allocate!(atmos, joinpath(ROOT_DIR, star_file))

    # Set deep heating if requested
    if deep_active
        AGNI.atmosphere.set_deep_heating!(atmos,
                                          active=true,
                                          P_dep=deep_P_dep,
                                          sigma_P=deep_sigma_P,
                                          efficiency=deep_efficiency)
    end

    # Set initial temperature profile (isothermal + adiabatic)
    AGNI.setpt.isothermal!(atmos, tmp_surf)

    # Calculate layer properties
    AGNI.atmosphere.calc_layer_props!(atmos)

    # Calculate fluxes (no solve, just single evaluation)
    AGNI.energy.calc_fluxes!(atmos, true, false, true, false, true)

    return atmos
end

In [ ]:
# Run model WITHOUT deep heating
@info "Running model without deep heating..."
out_dir_off = joinpath(OUT_DIR_BASE, "no_deep_heating")
atmos_off = run_model(out_dir_off, false)
@info "Done"

In [ ]:
# Run model WITH deep heating
@info "Running model with deep heating..."
out_dir_on = joinpath(OUT_DIR_BASE, "with_deep_heating")
atmos_on = run_model(out_dir_on, true)
@info "Done"

In [ ]:
# Compare results
println("="^60)
println("COMPARISON OF RESULTS")
println("="^60)

# TOA fluxes
OLR_off = atmos_off.flux_u_lw[1]
OLR_on = atmos_on.flux_u_lw[1]

flux_tot_toa_off = atmos_off.flux_tot[1]
flux_tot_toa_on = atmos_on.flux_tot[1]

flux_tot_boa_off = atmos_off.flux_tot[end]
flux_tot_boa_on = atmos_on.flux_tot[end]

# Deep heating flux at BOA (should equal F_deep_expected)
flux_deep_boa = atmos_on.flux_deep[end]

@printf("\nWithout Deep Heating:\n")
@printf("  OLR (LW up at TOA):     %.2f W/m²\n", OLR_off)
@printf("  Total flux at TOA:      %.2f W/m²\n", flux_tot_toa_off)
@printf("  Total flux at BOA:      %.2f W/m²\n", flux_tot_boa_off)

@printf("\nWith Deep Heating (ε=%.2f):\n", deep_efficiency)
@printf("  OLR (LW up at TOA):     %.2f W/m²\n", OLR_on)
@printf("  Total flux at TOA:      %.2f W/m²\n", flux_tot_toa_on)
@printf("  Total flux at BOA:      %.2f W/m²\n", flux_tot_boa_on)
@printf("  Deep heating flux (BOA): %.2f W/m²\n", flux_deep_boa)

@printf("\nDifferences:\n")
@printf("  ΔOLR:                   %.2f W/m²\n", OLR_on - OLR_off)
@printf("  ΔTotal flux (TOA):      %.2f W/m²\n", flux_tot_toa_on - flux_tot_toa_off)
@printf("  ΔTotal flux (BOA):      %.2f W/m²\n", flux_tot_boa_on - flux_tot_boa_off)

@printf("\nExpected deep heating flux: %.2f W/m²\n", F_deep_expected)
@printf("Actual deep heating flux:   %.2f W/m²\n", flux_deep_boa)
@printf("Difference:                 %.4f W/m² (%.2f%%)\n",
        abs(flux_deep_boa - F_deep_expected),
        100*abs(flux_deep_boa - F_deep_expected)/F_deep_expected)

In [ ]:
# Plot temperature profiles
p1 = plot(xlabel="Temperature [K]", ylabel="Pressure [bar]",
          yflip=true, yscale=:log10, legend=:topright,
          title="Temperature Profile", size=(500, 400))

plot!(p1, atmos_off.tmp, atmos_off.p .* 1e-5,
      label="No Deep Heating", lw=2, color=:blue)
plot!(p1, atmos_on.tmp, atmos_on.p .* 1e-5,
      label="With Deep Heating", lw=2, color=:red, ls=:dash)

display(p1)

In [ ]:
# Plot flux profiles
p2 = plot(xlabel="Flux [W m⁻²]", ylabel="Pressure [bar]",
          yflip=true, yscale=:log10, legend=:topright,
          title="Total Flux Profile", size=(500, 400))

plot!(p2, atmos_off.flux_tot, atmos_off.pl .* 1e-5,
      label="Total (no deep)", lw=2, color=:blue)
plot!(p2, atmos_on.flux_tot, atmos_on.pl .* 1e-5,
      label="Total (with deep)", lw=2, color=:red)
plot!(p2, atmos_on.flux_deep, atmos_on.pl .* 1e-5,
      label="Deep Heating", lw=2, color=:magenta, ls=:dash)

# Mark deposition pressure
hline!(p2, [deep_P_dep * 1e-5], ls=:dot, color=:gray, label="P_dep")

display(p2)

In [ ]:
# Plot deep heating flux gradient (dF/dP profile)
# Calculate the flux gradient from the cumulative flux
dF_deep = diff(atmos_on.flux_deep)
dp = diff(atmos_on.pl)
dF_dP = dF_deep ./ dp

p3 = plot(xlabel="dF/dP [W m⁻² Pa⁻¹]", ylabel="Pressure [bar]",
          yflip=true, yscale=:log10, legend=:topright,
          title="Deep Heating Rate (dF/dP)", size=(500, 400))

plot!(p3, dF_dP, atmos_on.p .* 1e-5,
      label="dF_deep/dP", lw=2, color=:magenta)

# Mark deposition pressure
hline!(p3, [deep_P_dep * 1e-5], ls=:dot, color=:gray, label="P_dep")

display(p3)

In [ ]:
# Combined plot with all flux components
p4 = plot(xlabel="Flux [W m⁻²]", ylabel="Pressure [bar]",
          yflip=true, yscale=:log10, legend=:outerright,
          title="Flux Components (With Deep Heating)", size=(700, 450))

# Radiative fluxes
plot!(p4, atmos_on.flux_u_lw, atmos_on.pl .* 1e-5,
      label="LW Up", lw=2, color=:red)
plot!(p4, -atmos_on.flux_d_lw, atmos_on.pl .* 1e-5,
      label="LW Down", lw=2, color=:red, ls=:dash)
plot!(p4, atmos_on.flux_u_sw, atmos_on.pl .* 1e-5,
      label="SW Up", lw=2, color=:orange)
plot!(p4, -atmos_on.flux_d_sw, atmos_on.pl .* 1e-5,
      label="SW Down", lw=2, color=:orange, ls=:dash)

# Convective flux
plot!(p4, atmos_on.flux_cdry, atmos_on.pl .* 1e-5,
      label="Convection", lw=2, color=:blue)

# Deep heating flux
plot!(p4, atmos_on.flux_deep, atmos_on.pl .* 1e-5,
      label="Deep Heating", lw=3, color=:magenta)

# Total flux
plot!(p4, atmos_on.flux_tot, atmos_on.pl .* 1e-5,
      label="Total", lw=2, color=:black)

# Mark deposition pressure
hline!(p4, [deep_P_dep * 1e-5], ls=:dot, color=:gray, lw=1, label="P_dep")

display(p4)

In [ ]:
# Save combined comparison plot
p_combined = plot(p1, p2, p3, p4, layout=(2, 2), size=(1000, 800))
savefig(p_combined, joinpath(OUT_DIR_BASE, "deep_heating_comparison.png"))
@info "Saved comparison plot to $(joinpath(OUT_DIR_BASE, "deep_heating_comparison.png"))"
display(p_combined)

In [ ]:
# Clean up
AGNI.atmosphere.deallocate!(atmos_off)
AGNI.atmosphere.deallocate!(atmos_on)
@info "Test complete!"